In [19]:

import inspect
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_anthropic import ChatAnthropic
from langgraph.prebuilt import create_react_agent
import tools_utility as t_utility
import ore_xml_tools as tools_ore
from react_agent_system_prompts import ore_agent_system_prompt_content

import getpass
import os
import shutil


def copy_directory(src_dir):
    dst_dir= os.path.basename(src_dir)+'new'
    dst_dir = os.path.join(os.path.dirname(src_dir), dst_dir)

    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)
    for item in os.listdir(src_dir):
        s = os.path.join(src_dir, item)
        d = os.path.join(dst_dir, item)
        shutil.copy2(s, d)

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

_set_env("ANTHROPIC_API_KEY")



from langchain.tools import tool
import xml.etree.ElementTree as ET
from typing import Tuple




@tool(response_format="content_and_artifact")
def read_xml_file(file_path: str = "ore.xml") -> Tuple[str, ET.Element]:
    """
    Read and parse XML file from given file_path.

    Args:
        file_path (str): The path to the XML file. Defaults to 'ore.xml'.

    Returns:
        Tuple[str, ET.Element]: A tuple containing a string indicating if the file has been read successfully or not, and the parsed XML root object.
    """
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        return "File read successfully.", root
    except ET.ParseError:
        return "Error parsing XML file.", ET.Element()
    except FileNotFoundError:
        return "File not found.", ET.Element()

@tool(response_format="content")
def get_parameter_value(section: str, parameter_name: str, file_path: str = "ore.xml") -> str:
    """
    Retrieve the value of a specific parameter from the specified section in the given XML file.

    Args:
        section (str): The section name, e.g., 'Setup' or 'Markets'.
        parameter_name (str): The name of the parameter to retrieve.
        file_path (str): The path to the XML file. Defaults to 'ore.xml'.

    Returns:
        str: The value of the parameter, or an error message if not found.
    """
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        section_elem = root.find(section)
        if section_elem is None:
            return f"Section '{section}' not found."
        param_elem = section_elem.find(f"Parameter[@name='{parameter_name}']")
        if param_elem is not None:
            return param_elem.text
        else:
            return "Parameter not found."
    except ET.ParseError:
        return "Error parsing XML file."
    except FileNotFoundError:
        return "File not found."

@tool(response_format="content")
def set_parameter_value(root_path: str, section: str, parameter_name: str, new_value: str) -> str:
    """
    Set the value of a specific parameter in the specified section in the given XML file.

    Args:
        root_path (str): The path to the XML file.
        section (str): The section name, e.g., 'Setup' or 'Markets'.
        parameter_name (str): The name of the parameter to set.
        new_value (str): The new value to set for the parameter.
        root (ET.Element): The root element of the parsed XML tree.

    Returns:
        str: A confirmation message indicating success or failure.
    """
    try:
        root = ET.parse(root_path).getroot()
        section_elem = root.find(section)
        if section_elem is None:
            return f"Section '{section}' not found."
        param_elem = section_elem.find(f"Parameter[@name='{parameter_name}']")
        if param_elem is not None:
            param_elem.text = new_value
            tree = ET.ElementTree(root)
            tree.write(root_path)
            return f"Parameter '{parameter_name}' in section '{section}' set to '{new_value}'. saved_results to {root_path}"
        else:
            return "Parameter not found."
    except ET.ParseError:
        return "Error parsing XML file."
    except FileNotFoundError:
        return "File not found."




ore_tools_list = []#[obj for _, obj in inspect.getmembers(tools_ore) if inspect.isfunction(obj)]
utility_tools_list = [obj for _, obj in inspect.getmembers(t_utility) if inspect.isfunction(obj)]
all_tools_list = ore_tools_list + utility_tools_list
all_tools_list = [get_parameter_value,set_parameter_value,read_xml_file]










In [20]:
all_tools_list[0]

StructuredTool(name='get_parameter_value', description="Retrieve the value of a specific parameter from the specified section in the given XML file.\n\nArgs:\n    section (str): The section name, e.g., 'Setup' or 'Markets'.\n    parameter_name (str): The name of the parameter to retrieve.\n    file_path (str): The path to the XML file. Defaults to 'ore.xml'.\n\nReturns:\n    str: The value of the parameter, or an error message if not found.", args_schema=<class 'langchain_core.utils.pydantic.get_parameter_value'>, func=<function get_parameter_value at 0x000001F23AD51580>)

In [21]:
llm = ChatAnthropic(model="claude-3-5-haiku-latest", )

agent = create_react_agent(llm,
    tools = all_tools_list,
)

input_messages = {'messages': [SystemMessage(content=ore_agent_system_prompt_content), HumanMessage(content="What is the base currency in the ore.xml file? ")]}

print_stream(agent.stream(input_messages, stream_mode="values"))


================================ Human Message =================================

What is the base currency in the ore.xml file? 


AuthenticationError: Error code: 401 - {'type': 'error', 'error': {'type': 'authentication_error', 'message': 'invalid x-api-key'}}

In [22]:


test = llm.invoke('Hi How are you?')


AuthenticationError: Error code: 401 - {'type': 'error', 'error': {'type': 'authentication_error', 'message': 'invalid x-api-key'}}

In [21]:
copy_directory(r'D:\Project_H\Examples\Example_1\Input')

In [23]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(
    model="claude-3-5-haiku-latest",
    temperature=0,
    max_tokens=1024,
    timeout=None,
    max_retries=2,
    # other params...
)

In [24]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AuthenticationError: Error code: 401 - {'type': 'error', 'error': {'type': 'authentication_error', 'message': 'invalid x-api-key'}}